In [1]:
import random

import numpy as np
import pandas as pd
from scipy import sparse
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, log_loss
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.preprocessing import OneHotEncoder, StandardScaler

Я сделал задание в .py файле (так было удобнее его запускать каждый раз + юпитер ноутбук не справляется с освобождением памяти после загрукзи датасета 1 раз в память)

Сюда просто переношу фрагментами и объясняю, что они делают.

Также, в разных частях кода я использую не весь датасет, а его часть - это связано с тем, что кросс-валидацию и другие вещи на всём датасете делать очень долго, и у меня не хватает памяти (16гб оперативки), чтобы делать это параллельно

Зафиксируем рандом, чтобы добиться воспроизводимости результатов

In [2]:
random.seed(1337)
np.random.seed(1337)

В качестве модели возьмём логистическую регрессию. В ней можно использовать разные оптимизаторы, если мы хотим не sgd и работать на больших данных, то подходит lbfgs и newton-cg, так как указано, что они лучше сходятся для большого объема данных. На деле лучше сходится liblinear, его и используем

In [3]:
def get_model(C=0.1):
    return LogisticRegression(penalty='l2',
                              C=C,
                              solver='liblinear',
                              random_state=1337,
                              max_iter=100,
                              verbose=0
                              )

Функция для препроцессинга

Время в чистом виде использовать сложно. Скорее всего, на клик может влиять день недели (например, в пятницу-воскресенье все расслабляются и охотнее покупают), и час (днём, на работе, некогда отвлекаться на рекламу, но вечером и дома вполне можно). Поэтому я добавляю фичи день недели и час. Однако я померил и особо крутого прироста они не дают

campaing_clicks единственная фича, которую можно интерпретироавть как числовую - вероятно, чем меньше campagin_click, тем лучше реклама. Поэтому её надо отнормализовать

Остальные фичи нужно one-hot-encodить. Получится на самом деле очень много фич, больше 4000

In [4]:
ohe = None # OneHotEncoder должен быть обучен по всему датасету, так что мы его запомним

In [5]:
def preprocess(X, y, remember=False, return_last_day=False):
    global ohe

    X['date_time'] = pd.to_datetime(X['date_time'])

    if return_last_day:
        idx = X['date_time'].dt.date == X['date_time'].dt.date.max()

    X['day_of_week'] = X['date_time'].dt.dayofweek
    X['hour'] = X['date_time'].dt.hour

    X = X.drop(columns=['date_time'])

    to_normalize = ['campaign_clicks']
    to_one_hot = ['zone_id', 'banner_id', 'os_id', 'country_id', 'day_of_week', 'hour']

    X[to_normalize] = StandardScaler().fit_transform(X[to_normalize])

    X_to = X[to_one_hot]

    if ohe is None:
        ohe = OneHotEncoder()
        X_to = ohe.fit_transform(X_to)
        if not remember:
            ohe = None
    else:
        X_to = ohe.transform(X_to)


    X = X.drop(columns=to_one_hot)
    X = sparse.hstack([X, X_to])
    X = X.tocsr()

    if return_last_day:
        X_last, y_last = X[idx], y[idx]
        return X, y, X_last, y_last
    else:
        return X, y

Просто функция, которая достаёт и препроцессит нужное количество данных, опционально шаффлит

При этом можно заметить, что некоторые колонки вообще не нужны: impressions всегда 1 (реклама в датасете всегда была показана), так что можно её выкинуть

In [6]:
def get_data(nrows=None, remember=False, shuffle=True, return_last_day=False):
    df = pd.read_csv('/opt/downloads/ml/recsys/data.csv',
                     nrows=nrows,
                     usecols=[
                         'date_time',
                         'zone_id',
                         'banner_id',
                         'campaign_clicks',
                         'os_id',
                         'country_id',
                         'clicks'
                     ])

    if shuffle:
        df = df.sample(frac=1).reset_index(drop=True)

    X, y = df.drop(columns=['clicks']), df['clicks']

    return preprocess(X, y, remember=remember, return_last_day=return_last_day)

Фич получилось многовато. Давайте сделаем feature selection и отсечем фичи с низкой важностью

In [7]:
def select_features(validate=False):
    X, y = get_data(1000000, shuffle=False)

    model = get_model()

    fs = SelectFromModel(model)
    fs.fit(X, y)

    if validate:
        X, y = get_data(1000000, shuffle=False)
        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8)
        model_full, model_sel = get_model(), get_model()

        model_full.fit(X_train, y_train)
        model_sel.fit(fs.transform(X_train), y_train)
        print(model_full.score(X_test, y_test))
        print(model_sel.score(fs.transform(X_test), y_test))

    return fs

Займёмся регуляризацией. Попробуем разные C и сделаем дополнительно кросс-валидацию

In [8]:
def adjust_regularization():
    X, y = get_data(1000000, shuffle=False)

    metrics = ['neg_log_loss', 'roc_auc']

    Cs = [0.001, 0.01, 0.1, 1, 10]
    nlls = []
    print('Regularization')
    for C in Cs:
        model = get_model(C=C)
        cv = cross_validate(model, X, y, cv=5, n_jobs=5, return_estimator=True, scoring=metrics, verbose=0)

        print(f'C = {C}')
        for i, metric in enumerate(metrics):
            metric_values = cv[f"test_{metric}"]
            print(f'{metric}: avg: {np.average(metric_values)}, metrics: {metric_values}')
        nlls.append(np.average(cv[f"test_neg_log_loss"]))
    return Cs[np.argmax(nlls)]

Наконец запустим наш код

In [9]:
X_train, y_train, X_test, y_test = get_data(None, remember=True, shuffle=True, return_last_day=True)

fs = select_features()
print(f'Selected {np.sum(fs.get_support())}/{len(fs.get_support())} features')
X_train = fs.transform(X_train)
X_test = fs.transform(X_test)

C = adjust_regularization()
print(f'Selected {C} for regularization')

model = get_model(C=C)

model.fit(X_train, y_train)

y_test_pred = model.predict(X_test)
auc = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])
ll = log_loss(y_test, model.predict_proba(X_test)[:, 1])
print(f'Auc: {auc}, ll: {ll}')

y_test_pred_baseline = np.full(y_test_pred.shape, np.mean(y_train))
auc_baseline = roc_auc_score(y_test, y_test_pred_baseline)
ll_baseline = log_loss(y_test, y_test_pred_baseline)
print(f'Auc: {auc_baseline}, ll: {ll_baseline}')

Selected 1437/5137 features
Regularization
C = 0.001
neg_log_loss: avg: -0.680740132490651, metrics: [-0.84411357 -0.66685434 -0.64370331 -0.61320486 -0.63582459]
roc_auc: avg: 0.6126805811728866, metrics: [0.42206964 0.62120583 0.65443841 0.69926418 0.66642484]
C = 0.01
neg_log_loss: avg: -0.7080524329656097, metrics: [-1.01587563 -0.65578927 -0.62851286 -0.60638804 -0.63369636]
roc_auc: avg: 0.6215720009055564, metrics: [0.39627361 0.6485326  0.68049035 0.70801284 0.6745506 ]
C = 0.1
neg_log_loss: avg: -0.7251968199238948, metrics: [-1.1081896  -0.65190166 -0.62514571 -0.6078433  -0.63290382]
roc_auc: avg: 0.6274334783727794, metrics: [0.40571681 0.65757954 0.68653014 0.70813095 0.67920994]
C = 1
neg_log_loss: avg: -0.7292812735916836, metrics: [-1.12759203 -0.65129937 -0.62507155 -0.60913393 -0.63330949]
roc_auc: avg: 0.6298197225877247, metrics: [0.41244255 0.66028188 0.68792361 0.70787271 0.68057787]
C = 10
neg_log_loss: avg: -0.7305484660254434, metrics: [-1.13131221 -0.65187174 

Бейзлайн превзойдён!